# Baseline
Setting up a more robust baseline notebook, suitable for use with all of the "Big Three" (XGBoost, CatBoost, LightGBM) libraries and on either Google Colab or the local machine.

# Setup

In [1]:
# two manual flags (ex-config)
colab = False
gpu_available = True
# libraries = ['xgboost', 'lightgbm', 'catboost']
libraries = ['fastai']

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime

In [3]:
%matplotlib inline
%config Completer.use_jedi = False
os.environ['WANDB_NOTEBOOK_NAME'] = f"widedeep_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# handle Google Colab-specific library installation/updating
if colab:
    # much of the below inspired by or cribbed from the May 2021 Kaggle Tabular Playground winner, at 
    # https://colab.research.google.com/gist/academicsuspect/0aac7bd6e506f5f70295bfc9a3dc2250/tabular-may-baseline.ipynb?authuser=1#scrollTo=LJoVKJb5wN0L
    
    # Kaggle API for downloading the datasets
    !pip install --upgrade -q kaggle

    # weights and biases
    !pip install -qqqU wandb
    
    # Optuna for parameter search
    !pip install -q optuna

    # upgrade sklearn
    !pip install --upgrade scikit-learn

    !pip install category_encoders
    
    if 'catboost' in libraries:
        !pip install catboost
    
    if 'xgboost' in libraries:
        if gpu_available: 
            # this part is from https://github.com/rapidsai/gputreeshap/issues/24
            !pip install cmake --upgrade
            # !pip install sklearn --upgrade
            !git clone --recursive https://github.com/dmlc/xgboost
            %cd /content/xgboost
            !mkdir build
            %cd build
            !cmake .. -DUSE_CUDA=ON
            !make -j4
            %cd /content/xgboost/python-package
            !python setup.py install --use-cuda --use-nccl
            !/opt/bin/nvidia-smi
            !pip install shap
        else:
            !pip install --upgrade xgboost
    if 'lightgbm' in libraries:
        if gpu_available:
            # lighgbm gpu compatible
            !git clone --recursive https://github.com/Microsoft/LightGBM
            ! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;
        else:
            !pip install --upgrade lightgbm
        

        

Now, non-stdlib imports

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

# general ML tooling
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import log_loss, roc_auc_score
import wandb
# from wandb.xgboost import wandb_callback
# from wandb.lightgbm import wandb_callback
from sklearn.impute import SimpleImputer #, KNNImputer
# import timm

import seaborn as sns

# from catboost import CatBoostClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
from fastai.tabular.all import *
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.preprocessing import StandardScaler #, MinMaxScaler, MaxAbsScaler, RobustScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectKBest, f_regression
from joblib import dump, load
# feature engineering tools
# from sklearn.feature_selection import mutual_info_regression
# import featuretools as ft


/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /tmp/pip-req-build-1_ic8ial/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [6]:
torch.cuda.is_available()

False

Now, datapath setup

In [7]:
# # This is the code for reading the train.csv and converting it to a .feather file
# df = pd.read_csv(datapath/'train.csv', index_col='id', low_memory=False)
# df.index.name = None
# df.to_feather(path='./dataset_df.feather')

In [8]:
if colab:
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/sep2021/')
    
else:
    # if on local machine
    datapath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/sep2021/')    
    


## Ex-Model Config

In [9]:
# meta-config for preprocessing and cross-validation, but NOT for model parameters
exmodel_config = {
    # model config
#     "model": XGBClassifier,
#     "n_estimators": 100, 
#     "max_depth": 3,
#     "learning_rate": 0.1,
#     "test_size": 0.2,
#     "reg_lambda": None, 
    "library": 'widedeep',
    "scaler": StandardScaler, # TODO: experiment with others (but imputation may be slow)
    "scale_b4_impute": False,
    "imputer": SimpleImputer(strategy='median', add_indicator=True),
#     "knn_imputer_n_neighbors": None, # None if a different imputer is used
#     "feature_selector": SelectKBest,
#     "k_best": 80,
#     "feature_selection_scoring": f_regression,
    'random_state': 42,
    'feature_generation': ['NaN_counts', 'SummaryStats', 'NaN_OneHots'],
    'features_categorized': True,
#     'subsample': 1,
#     'cross_val_strategy': KFold, # None for holdout, or the relevant sklearn class
#     'kfolds': 5, # if 1, that means just doing holdout
#     'test_size': 0.2,
#     'features_created': False,
#     'feature_creator': None,
}

## Data Setup

Due to the importance of identifying categorical variables for deep learning on tabular data (namely, the generation of embeddings containing meaningful information about them), I'm going to try using `fastai`'s `cont_cat_split` on the original dataset (post-imputation and generation of features based on summary statistics) and then proceeding with the other transforms.

In [10]:
# # here's how to load the original, unaltered dataset and separate features from targets
# df = pd.read_feather(path=datapath/'dataset_df.feather') # this is the unaltered original dataset
# features = [x for x in df.columns if x != 'claim']
# X = df[features]
# y = df.claim



# load the version of the dataset with imputations; X and y were stored separately, as feather and joblib respectively
X = pd.read_feather(datapath/'X_NaNcounts_SummaryStats_imputed-Median-wIndicators-StandardScaled.feather') 
y = load(datapath/'y.joblib')    
X.index.name = 'id'
y.index.name = 'id'

exmodel_config['feature_count'] = len(X.columns)


In [11]:
len(X.columns)

246

In [12]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [13]:
X.nunique(axis=0)

0      147965
1      117427
2      274624
3      265618
4      125414
5      286473
6      201975
7      209765
8      368026
9      370266
10     157265
11     325095
12     117373
13     231413
14     308572
15     313089
16      77994
17     172559
18     160073
19     318317
20     313905
21      83438
22     148460
23     176988
24     208329
25     333767
26     290999
27     269916
28     233936
29     135072
30     263378
31     297891
32     281103
33     245721
34     357008
35     316292
36     210948
37      38267
38     229893
39     223437
40     307516
41     264543
42      81540
43     213949
44     245352
45      84044
46     212671
47      87459
48     245196
49     127050
50     305728
51     212595
52     342517
53       7459
54     117101
55     240290
56     246959
57     163484
58      76378
59     158582
60     245796
61     266661
62     194153
63     267355
64     142378
65     149027
66     213419
67     209319
68      14049
69     210462
70      30980
71    

I'll follow JH's advice here and set 10k as the ceiling for the `max_card=`; this might be something to experiment with later using Optuna. (Esp. as JH says that numbers in the 5000s and higher make him very nervous.)

In [14]:
max_card_embed = 10000
max_card_cat = 100000
exmodel_config['max_card_for_embedding'] = max_card_embed
exmodel_config['max_card_for_categorical'] = max_card_cat


# X_orig = X.iloc[:, :118] # excluding summary, meta-statistics
# X_meta = X.iloc[:, 118:] # including summary, meta-statistics

In [15]:
exmodel_config['max_card_for_categorical']

100000

In [16]:
low_card_features = [f for f in X.columns if X[f].nunique() <= 50000]
high_card_features = [f for f in X.columns if X[f].nunique() > 50000]

In [17]:
len(low_card_features)

129

In [18]:
len(high_card_features)

117

# WideDeep

## (Example)

In [19]:
# df = pd.read_csv(datapath/"adult.csv.zip")
# df["income_label"] = (df["income"].apply(lambda x: ">50K" in x)).astype(int)
# df.drop("income", axis=1, inplace=True)
# df_train, df_test = train_test_split(df, test_size=0.2, stratify=df.income_label)

In [20]:
# for f in df.columns:
#     print(f"{f}: {df[f].nunique()}")
#     print(f"NaNs: {df[f].isna().sum()}\n")

In [21]:
from pytorch_widedeep import Trainer
from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
from pytorch_widedeep.models import Wide, TabMlp, WideDeep
from pytorch_widedeep.metrics import Accuracy

In [22]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957919 entries, 0 to 957918
Columns: 246 entries, 0 to 245
dtypes: float64(246)
memory usage: 1.8 GB


/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
wide_cols_pre = [f for f in X.columns if X[f].nunique() <= max_card_cat and X[f].nunique() > 2]
wide_cols_onehot = [f for f in X.columns if X[f].nunique() == 2]
cont_cols = high_card_features
embed_cols = [f for f in X.columns if X[f].nunique() <= max_card_embed and X[f].nunique() > 2]

In [24]:
len(wide_cols_pre), len(wide_cols_onehot), len(cont_cols), len(embed_cols)

(18, 118, 117, 4)

In [25]:
wide_preprocessor = WidePreprocessor(wide_cols=wide_cols_pre)
X_wide_pre = wide_preprocessor.fit_transform(X)



In [26]:
X_wide_pre.shape

(957919, 18)

In [27]:
X_wide_pre[:10,:]

array([[     1,  77995, 161433, 199700, 281240, 365284, 452743, 460202,
        536580, 550629, 581609, 660306, 693792, 694221, 736126, 780964,
        793934, 793949],
       [     2,  77996, 161434, 199701, 281241, 365285, 452744, 460203,
        536581, 550630, 581610, 660307, 693793, 694222, 736127, 780965,
        793935, 793950],
       [     3,  77997, 161435, 199702, 281242, 365286, 452745, 460204,
        536582, 550631, 581611, 660308, 693794, 694223, 736128, 780966,
        793936, 793951],
       [     4,  77998, 161436, 199703, 281243, 365287, 452746, 460205,
        536583, 550632, 581612, 660309, 693795, 694224, 736129, 780967,
        793937, 793952],
       [     5,  77999, 161437, 199704, 281244, 365288, 452747, 460206,
        536584, 550633, 581613, 660310, 693794, 694225, 736130, 780968,
        793938, 793953],
       [     6,  78000, 161438, 199705, 281245, 365289, 452748, 460207,
        536585, 550634, 581614, 660311, 693796, 694226, 736131, 780969,
        793

In [28]:
X_wide_pre_df = pd.DataFrame(X_wide_pre)

In [29]:
X_wide_pre_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1,77995,161433,199700,281240,365284,452743,460202,536580,550629,581609,660306,693792,694221,736126,780964,793934,793949
1,2,77996,161434,199701,281241,365285,452744,460203,536581,550630,581610,660307,693793,694222,736127,780965,793935,793950
2,3,77997,161435,199702,281242,365286,452745,460204,536582,550631,581611,660308,693794,694223,736128,780966,793936,793951
3,4,77998,161436,199703,281243,365287,452746,460205,536583,550632,581612,660309,693795,694224,736129,780967,793937,793952
4,5,77999,161437,199704,281244,365288,452747,460206,536584,550633,581613,660310,693794,694225,736130,780968,793938,793953


In [30]:
X.loc[:, wide_cols_onehot].head()

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.127178,-0.126936,-0.128208,-0.128498,-0.127846,-0.128334,-0.128263,-0.127711,-0.127186,-0.127076,-0.12793,-0.128637,-0.128095,-0.127072,-0.128284,-0.12801,-0.127939,-0.127508,-0.128137,-0.128057,-0.128052,-0.127309,-0.127639,-0.128792,-0.128271,-0.127648,-0.12801,-0.127254,-0.127888,-0.127791,-0.128992,-0.128368,-0.128212,-0.127182,-0.127546,-0.127669,-0.127453,-0.127968,-0.128494,-0.12769,-0.127808,-0.127892,-0.128057,-0.128090,-0.128174,-0.128804,-0.128343,-0.128019,-0.12774,-0.128506,7.814957,-0.127466,-0.128086,-0.12793,-0.127918,-0.128107,-0.128553,-0.128057,-0.127825,-0.128498,-0.127956,-0.128322,-0.127867,-0.128574,-0.127884,-0.127884,-0.128187,-0.128745,-0.128464,-0.127242,-0.12817,-0.127059,-0.128401,-0.12854,-0.128061,-0.128536,-0.127195,-0.128031,-0.127732,-0.127487,-0.127597,-0.128183,-0.128779,-0.127762,-0.128031,-0.128343,-0.12747,-0.128448,-0.128015,-0.12817,-0.128275,-0.128212,-0.128065,-0.127884,-0.128662,-0.127339,-0.127254,-0.127352,-0.127968,-0.128355,-0.12761,-0.126843,-0.128666,-0.12697,-0.127766,-0.128431,-0.127757,-0.127572,-0.128368,-0.127677,-0.128242,-0.127867,-0.127119,-0.127985,-0.128494,-0.12862,-0.127854,-0.12703
1,-0.127178,-0.126936,-0.128208,-0.128498,-0.127846,-0.128334,-0.128263,-0.127711,-0.127186,-0.127076,-0.12793,-0.128637,-0.128095,-0.127072,-0.128284,-0.12801,-0.127939,-0.127508,-0.128137,-0.128057,-0.128052,-0.127309,-0.127639,-0.128792,-0.128271,-0.127648,-0.12801,-0.127254,-0.127888,-0.127791,-0.128992,-0.128368,-0.128212,-0.127182,-0.127546,-0.127669,-0.127453,-0.127968,-0.128494,-0.12769,-0.127808,-0.127892,-0.128057,-0.128090,-0.128174,-0.128804,-0.128343,-0.128019,-0.12774,-0.128506,-0.127960,-0.127466,-0.128086,-0.12793,-0.127918,-0.128107,-0.128553,-0.128057,-0.127825,-0.128498,-0.127956,-0.128322,-0.127867,-0.128574,-0.127884,-0.127884,-0.128187,-0.128745,-0.128464,-0.127242,-0.12817,-0.127059,-0.128401,-0.12854,-0.128061,-0.128536,-0.127195,-0.128031,-0.127732,-0.127487,-0.127597,-0.128183,-0.128779,-0.127762,-0.128031,-0.128343,-0.12747,-0.128448,-0.128015,-0.12817,-0.128275,-0.128212,-0.128065,-0.127884,-0.128662,-0.127339,-0.127254,-0.127352,-0.127968,-0.128355,-0.12761,-0.126843,-0.128666,-0.12697,-0.127766,-0.128431,-0.127757,-0.127572,-0.128368,-0.127677,-0.128242,-0.127867,-0.127119,-0.127985,-0.128494,-0.12862,-0.127854,-0.12703
2,-0.127178,-0.126936,-0.128208,-0.128498,-0.127846,-0.128334,-0.128263,-0.127711,-0.127186,-0.127076,-0.12793,-0.128637,-0.128095,-0.127072,7.795214,-0.12801,-0.127939,-0.127508,-0.128137,-0.128057,-0.128052,-0.127309,-0.127639,-0.128792,-0.128271,-0.127648,-0.12801,-0.127254,-0.127888,-0.127791,-0.128992,-0.128368,-0.128212,-0.127182,-0.127546,-0.127669,7.846019,-0.127968,-0.128494,-0.12769,-0.127808,-0.127892,-0.128057,-0.128090,-0.128174,-0.128804,-0.128343,-0.128019,-0.12774,-0.128506,-0.127960,-0.127466,-0.128086,-0.12793,-0.127918,-0.128107,-0.128553,-0.128057,-0.127825,-0.128498,-0.127956,-0.128322,-0.127867,-0.128574,-0.127884,-0.127884,-0.128187,-0.128745,7.784252,-0.127242,-0.12817,-0.127059,-0.128401,-0.12854,-0.128061,-0.128536,-0.127195,-0.128031,-0.127732,-0.127487,-0.127597,-0.128183,-0.128779,-0.127762,-0.128031,-0.128343,-0.12747,-0.128448,-0.128015,7.80212,-0.128275,-0.128212,-0.128065,-0.127884,-0.128662,-0.127339,7.858285,-0.127352,-0.127968,-0.128355,-0.12761,-0.126843,-0.128666,-0.12697,-0.127766,-0.128431,-0.127757,-0.127572,-0.128

In [31]:
X_wide = X_wide_pre_df.join(X.loc[:,wide_cols_onehot])

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:
X_wide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957919 entries, 0 to 957918
Columns: 136 entries, 0 to 245
dtypes: float64(118), int64(18)
memory usage: 993.9 MB


In [33]:
wide = Wide(wide_dim=np.unique(X_wide).shape[0], pred_dim=1)

In [34]:
tab_preprocessor = TabPreprocessor(embed_cols=embed_cols, continuous_cols=cont_cols)

In [35]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957919 entries, 0 to 957918
Columns: 246 entries, 0 to 245
dtypes: float64(246)
memory usage: 1.8 GB


In [36]:
X_tab = tab_preprocessor.fit_transform(X)

In [37]:
X_tab.shape

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(957919, 121)

In [38]:
deeptabular = TabMlp(
    mlp_hidden_dims=[64,32],
    column_idx=tab_preprocessor.column_idx,
    embed_input=tab_preprocessor.embeddings_input,
    continuous_cols=cont_cols,
)

In [39]:
model = WideDeep(wide=wide, deeptabular=deeptabular)

In [42]:
X_wide = np.array(X_wide)

In [44]:
X_wide.shape

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(957919, 136)

In [45]:
X_tab.shape

(957919, 121)

39774, 758737, 552968

In [52]:
trainer = Trainer(model, objective='binary', metrics=[Accuracy], seed=42)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [56]:
y = np.array(y)

In [57]:
trainer.fit(
    X_wide=X_wide,
    X_tab=X_tab,
    target=y,
    n_epochs=5,
    batch_size=1024,
    val_split=0.2,
)

valid: 100%|██████████| 188/188 [00:01<00:00, 112.74it/s, loss=0.924, metrics={'acc': 0.7447}]


In [62]:
X_test = pd.read_feather(datapath/'X_test_NaNcounts_SummaryStats_imputed-Median-wIndicators-StandardScaled.feather')

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [72]:
X_test = X_test.to_numpy()
X_wide_te = wide_preprocessor.transform(X_test)
X_tab_te = tab_preprocessor.transform(X_test)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/pytorch_widedeep/preprocessing/wide_preprocessor.py:136: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return df.copy()[self.wide_cols]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [73]:
X_test.shape

(493474, 246)

In [75]:
X_test = pd.read_feather(datapath/'X_test_NaNcounts_SummaryStats_imputed-Median-wIndicators-StandardScaled.feather')

low_card_features = [f for f in X_test.columns if X_test[f].nunique() <= 50000]
high_card_features = [f for f in X_test.columns if X_test[f].nunique() > 50000]

wide_cols_pre = [f for f in X_test.columns if X_test[f].nunique() <= max_card_cat and X_test[f].nunique() > 2]
wide_cols_onehot = [f for f in X_test.columns if X_test[f].nunique() == 2]
cont_cols = high_card_features
embed_cols = [f for f in X_test.columns if X_test[f].nunique() <= max_card_embed and X_test[f].nunique() > 2]

In [78]:
# X_test = X_test.to_numpy()
X_wide_te = wide_preprocessor.transform(X_test)
X_tab_te = tab_preprocessor.transform(X_test)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
X_wide_te = wide_preprocessor

In [ ]:
preds = trainer.predict(X_wide=X_wide_te, X_tab=X_tab_te)

In [ ]:
preds[:20]

In [ ]:
preds_proba = trainer.predict_proba(X_wide=X_wide_te, X_tab=X_tab_te)

In [ ]:
preds_proba[:20]

In [ ]:
dump(preds_proba, datapath/'preds/widedeep_5epochs_bs1024_64x32tabmlp_20210929_probas.joblib')

## Weights and Biases Run Config

Below is the configuration for a Weights and Biases (`wandb`) run.

In [15]:
# wandb config:
config_run = {
    'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
    'tags': ['stacking-sklearn', 'attempt'],
    'notes': "Trying a fastai tabular MLP model, for ensembling with the GBMs",
}

# Training

In [ ]:
learner.LR

In [23]:
def train(X_train, X_valid, y_train, y_valid, model_config, 
                                              random_state=42,
                                              exmodel_config=exmodel_config, 
                                              config_run=config_run):#, scaler): # passed in via config dict for now
    """
    Basic training function. Note that some of the options passed via the argument are
    in fact hard-coded in, to avoid inconveniences.
    :param X_train: the training set features
    :param X_valid: the validation set features
    :param y_train: the training set targets
    :param y_valid: the validation set targets
    :param random_staKFold: for reproducibility
    :param exmodel_config: dict containing configuration details including the library 
                            (thus model) used, preprocessing, and cross-validation
    :param model_config: dict containing hyperparameter specifications for the model
    :param config_run: dict containing wandb run configuration (name, etc)
    """
    
    # As of 20210920, best CatBoost config is:
    best_20210920_catboost_params = {
        'iterations': 3493,
        'depth': 5,
        'learning_rate': 0.09397459954141321,
        'random_strength': 43,
        'l2_leaf_reg': 26,
        'border_count': 239,
        'bagging_temperature': 12.532400413798356,
        'od_type': 'Iter'
    }
    
    # catboost 20210921 on colab (only 15 trials though)
    best_catboost_params = {
        'iterations': 3302,
        'depth': 5,
        'learning_rate': 0.017183208677599107,
        'random_strength': 41,
        'l2_leaf_reg': 30,
        'border_count': 251,
        'bagging_temperature': 9.898390369028036, 
        'od_type': 'IncToDec'
    }
    
    # optuna 20210921
    best_xgboost_params = {
        'n_estimators': 1119,
        'max_depth': 6,
        'learning_rate': 0.04123392555159452,
        'reg_alpha': 4.511876752318655,
        'reg_lambda': 4.074347238862406,
        'subsample': 0.8408586950521992
    }
    
    wandb.init(
        project="202109_Kaggle_tabular_playground",
        save_code=True,
        tags=config_run['tags'],
        name=config_run['name'],
        notes=config_run['notes'],
        config=exmodel_config)   
        
    if exmodel_config['library'] == 'xgboost':
        model = XGBClassifier(
            tree_method=model_config['tree_method'],
            random_state=random_state,
            n_jobs=model_config['n_jobs'], 
            verbosity=model_config['verbosity'], 
            objective=model_config['objective'],
            **best_xgboost_params
            # #             eval_metric=model_config['eval_metric'],

            # comment out the below for a fairly default model
#             booster=model_config['booster'],
#             max_depth=model_config['max_depth'],
#             learning_rate=model_config['learning_rate'], 
#             subsample=model_config['subsample'],
#             reg_alpha=model_config['reg_alpha'],
#             reg_lambda=model_config['reg_lambda'],
#             n_estimators=model_config['n_estimators'],
        )
        
        model.fit(X_train, y_train, callbacks=[wandb.xgboost.wandb_callback()],
#                                     eval_metric=model_config['eval_metric'],
                 )


    elif exmodel_config['library'] == 'lightgbm':
        model = LGBMClassifier(
#             boosting_type=model_config['boosting_type'],
#             max_depth=model_config['max_depth']
            # TODO
            random_state=random_state,
            n_jobs=model_config['n_jobs'],
            objective=model_config['objective'],
#             eval_metric=model_config['eval_metric'],
            boosting_type=model_config['boosting_type'],
            device_type=model_config['device_type'],
            
            # comment out the below for a basically default model
            n_estimators=model_config['n_estimators'],
            learning_rate=model_config['learning_rate'],
            max_depth=model_config['max_depth'],
            reg_alpha=model_config['reg_alpha'],
            reg_lambda=model_config['reg_lambda'],
            subsample=model_config['subsample'],
        )
        
        model.fit(X_train, y_train, callbacks=[wandb.lightgbm.wandb_callback()],
#                                     eval_metric=model_config['eval_metric'],
                 )
        
    elif exmodel_config['library'] == 'catboost':
        print("CatBoost, therefore no WandB callback.")
        model = CatBoostClassifier(
#             n_estimators=config['n_estimators'],
#             learning_rate=config['learning_rate'],
#             max_depth=config['max_depth'],
            task_type=model_config['task_type'],
    #         n_jobs=config['n_jobs'],
    #         verbosity=config['verbosity'],
    #         subsample=config['subsample'],
#             n_estimators=model_config['n_estimators'],
            random_state=random_state,
            # objective='Logloss', # default, accepts only one
#             custom_metrics=model_config['custom_metrics'],
    #         bootstrap_type=config['bootstrap_type'],
    #         device:config['device']
            **best_catboost_params
        ) 
        model.fit(X_train, y_train)
        
#     y_train_pred = model.predict(X_train)
    y_train_pred = model.predict_proba(X_train)[:,1]

    train_loss = log_loss(y_train, y_train_pred)
    train_auc = roc_auc_score(y_train, y_train_pred)
    wandb.log({'train_loss': train_loss, 'train_auc': train_auc})

    if exmodel_config['library'] == 'catboost':
        print(model.get_all_params())
        wandb.log(model.get_all_params())
    else:
        wandb.log(model.get_params()) # logging model parameters, trying bare-invocation rather than params: model.get_params()
    
    # trying with predict_proba
    y_pred = model.predict_proba(X_valid)[:,1]
#     y_pred = model.predict(X_valid)

    valid_loss = log_loss(y_valid, y_pred)
    valid_auc = roc_auc_score(y_valid, y_pred)
    wandb.log({'valid_loss':valid_loss, 'valid_auc':valid_auc})
    print(f"Valid log-loss is {valid_loss}\nValid AUC is {valid_auc}")   
#     wandb.finish()   
    return model
    

In [24]:
def cross_validation(model_config, X=X, y=y, start_fold=0, exmodel_config=exmodel_config, random_state=42):
    """
    Function to handle model training process in the context of cross-validation -- via hold-out or via k-fold.
    If exmodel_config['cross_val_strategy'] == None, then any kfolds= input is ignored; otherwise, the number specified is used.
    
    :param kfolds: int specifying number of k-folds to use in cross-validation
    :param exmodel_config: dict containing general config including for cross-validation -- `kfold=1` implies hold-out
    """
    if exmodel_config['kfolds'] == 1:
        print("Proceeding with holdout")
        X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=exmodel_config['test_size'], 
                                                      random_state=random_state,
                                                     )
        model = train(X_train, X_valid, y_train, y_valid, exmodel_config=exmodel_config, 
                                                    model_config=model_config,
                                                    config_run=config_run)
        wandb.finish()
        
    else:
        X, y = X.to_numpy(), y.to_numpy()
        kfold = exmodel_config['cross_val_strategy'](n_splits=exmodel_config['kfolds'], shuffle=True, random_state=random_state)
        models = {}
        model_path = Path(datapath/f"models/{config_run['name']}_{exmodel_config['kfolds']}folds/")
        (model_path).mkdir(exist_ok=True)
        for fold, (train_ids, valid_ids) in enumerate(kfold.split(X,y)):
            if fold < start_fold:
                continue
            else:
                print(f"FOLD {fold}")
                print("---------------------------------------------------")
                X_train, X_valid = X[train_ids], X[valid_ids]
                y_train, y_valid = y[train_ids], y[valid_ids]
                model = train(X_train, X_valid, y_train, y_valid, exmodel_config=exmodel_config, 
                                                    model_config=model_config,
                                                    config_run=config_run)
                wandb.log({'fold': fold})
                models[fold] = model
                dump(model, Path(model_path/f"{exmodel_config['library']}_fold{fold}_model.joblib"))
                wandb.finish()
        return models
        

# Interface

## Runs

In [25]:
# library = 'xgboost'
# exmodel_config['library'] = library
# model_config = model_configurator(library)
# xgboost_models = cross_validation(model_config)

In [26]:
# for scaler in [StandardScaler, MaxAbsScaler, MinMaxScaler, RobustScaler]:
#     exmodel_config['scaler'] = scaler
#     scaler = scaler()
#     X_scaled = scaler.fit_transform(X)
#     X = pd.DataFrame(X_scaled, columns=X.columns)
#     exmodel_config['library'] = 'lightgbm'
#     model_config = model_configurator('lightgbm')
#     cross_validation(model_config)

In [27]:
# library = 'lightgbm'
# exmodel_config['library'] = library
# model_config = model_configurator(library)
# lightgbm_models = cross_validation(model_config)

# Stacking

## Via `sklearn.ensemble.StackingClassifier`

In [28]:
# xgboost_estimators = [(f'xgboost_fold{fold}', xgboost_models[fold]) for fold in range(5)]

In [29]:
# leaving this default for first try
# final_estimator = 

In [30]:
def stacker(estimators:dict, library:str, X=X, y=y): #, load_models:bool=False, load_path:Path=None):
    """
    A wrapper that will take a dict of the form {fold:int : model} and a string representing the library (for file-naming), 
    then run `sklearn.ensemble.StackingClassifier` with it, and save the stacked model afterward
    """
    estimators_list = [(f'{library}_fold{fold}', estimators[fold]) for fold in range(5)]
    blender = StackingClassifier(estimators=estimators_list,
                                 cv=5,
                                 stack_method='predict_proba',
                                 n_jobs=2,
                                 passthrough=False,
                                 verbose=1
                                )
    print(f"Starting fitting at {datetime.now().strftime('%Y%m%d_%H%M%S')}")
    blender.fit(X,y)
    print(f"Fitting complete at {datetime.now().strftime('%Y%m%d_%H%M%S')}")
    dump(blender, filename=datapath/f"models/{config_run['name']}_{exmodel_config['kfolds']}folds/{library}_stack.joblib")
    print(f"Blender model saved at {datetime.now().strftime('%Y%m%d_%H%M%S')}")
    return blender
    

In [31]:
# might encapsulate this in a new version of the above train function later
exmodel_config['ensemble'] = 'stacking'

wandb.init(
        project="202109_Kaggle_tabular_playground",
        save_code=True,
        tags=config_run['tags'],
        name=config_run['name'],
        notes=config_run['notes'],
        config=exmodel_config)   

random_state = exmodel_config['random_state'] # 42


# # optuna 20210921
# best_xgboost_params = {
#     'n_estimators': 1119,
#     'max_depth': 6,
#     'learning_rate': 0.04123392555159452,
#     'reg_alpha': 4.511876752318655,
#     'reg_lambda': 4.074347238862406,
#     'subsample': 0.8408586950521992
# }

# model_config = model_configurator('xgboost')
# xgboost_model = XGBClassifier(
#             tree_method=model_config['tree_method'],
#             random_state=random_state,
# #             n_jobs=model_config['n_jobs'], 
#             verbosity=model_config['verbosity'], 
#             objective=model_config['objective'],
#             **best_xgboost_params
#             # #             eval_metric=model_config['eval_metric'],

#             # comment out the below for a fairly default model
# #             booster=model_config['booster'],
# #             max_depth=model_config['max_depth'],
# #             learning_rate=model_config['learning_rate'], 
# #             subsample=model_config['subsample'],
# #             reg_alpha=model_config['reg_alpha'],
# #             reg_lambda=model_config['reg_lambda'],
# #             n_estimators=model_config['n_estimators'],
#         )

# model_config = model_configurator('lightgbm')
# lightgbm_model = LGBMClassifier(
#             random_state=random_state,
# #             n_jobs=model_config['n_jobs'],
#             objective=model_config['objective'],
#             boosting_type=model_config['boosting_type'],
#             device_type=model_config['device_type'],
            
#             # comment out the below for a basically default model
#             n_estimators=model_config['n_estimators'],
#             learning_rate=model_config['learning_rate'],
#             max_depth=model_config['max_depth'],
#             reg_alpha=model_config['reg_alpha'],
#             reg_lambda=model_config['reg_lambda'],
#             subsample=model_config['subsample'],
#         )

model_config = model_configurator('catboost', gpu_available=False) # set GPU false to avoid parallel threads blocking GPU

# # As of 20210920, best CatBoost config is:
# best_20210920_catboost_params = {
#     'iterations': 3493,
#     'depth': 5,
#     'learning_rate': 0.09397459954141321,
#     'random_strength': 43,
#     'l2_leaf_reg': 26,
#     'border_count': 239,
#     'bagging_temperature': 12.532400413798356,
#     'od_type': 'Iter'
# }

# catboost 20210921 on colab (only 15 trials though)
best_catboost_params = {
    'iterations': 3302,
    'depth': 5,
    'learning_rate': 0.017183208677599107,
    'random_strength': 41,
    'l2_leaf_reg': 30,
    'border_count': 251,
    'bagging_temperature': 9.898390369028036, 
    'od_type': 'IncToDec'
}
    

catboost_model = CatBoostClassifier(
            task_type=model_config['task_type'],
#             n_estimators=model_config['n_estimators'],
            random_state=random_state,
            
            **best_catboost_params
        ) 



estimators_list = [
#     ('xgboost', xgboost_model),
#     ('lightgbm', lightgbm_model),
    ('catboost', catboost_model)
]

# wandb.log({'estimators': estimators_list})

final_estimator = LogisticRegression(max_iter=1000)
exmodel_config['blender_final_estimator'] = str(final_estimator)
exmodel_config['blender-passthrough'] = False

blender = StackingClassifier(estimators=estimators_list,
                             final_estimator=final_estimator,
                             cv=5,
                             stack_method='predict_proba',
                             n_jobs=-1, # 4 is max allowable for CPU
                             passthrough=exmodel_config['blender-passthrough'],
                             verbose=1
                            )


           
    

wandb: Currently logged in as: hushifang (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [32]:
wandb.log({'blender-final_estimator': str(blender.final_estimator),
#            'blender-final_estimator_params': str(blender.final_estimator.get_params()),
           'blender-stack_mdethod': 'predict_proba',
           'blender-cv': 5
          })

In [ ]:
print(f"Starting fitting at {datetime.now().strftime('%Y%m%d_%H%M%S')}")
blender.fit(X,y) # unsure of this -- given kwarg cv=5, is it producing the splits? Or do I have to somehow?
print(f"Fitting complete at {datetime.now().strftime('%Y%m%d_%H%M%S')}")

Starting fitting at 20210922_112912


In [ ]:
# wandb.log({'xgboost_params':str(blender.estimators[0][1].get_params()),
#            'lightgbm_params':str(blender.estimators[1][1].get_params()),
# #            'catboost_params':str(blender.estimators[2][1].get_all_params()),
#           })

In [ ]:
model_path = Path(datapath/f"models/{config_run['name']}/")
(model_path).mkdir(exist_ok=True)
dump(blender, filename=model_path/f"{config_run['name']}_stack.joblib")
print(f"Blender model saved at {datetime.now().strftime('%Y%m%d_%H%M%S')}")

In [ ]:
train_preds = blender.predict_proba(X)[:,1]
train_loss = log_loss(y_pred=train_preds, y_true=y)
train_auc = roc_auc_score(y, train_preds)
wandb.log({'train_loss': train_loss, 'train_auc': train_auc})
print(f"train_loss is {train_loss}, train_auc is {train_auc}")

In [ ]:
# train_preds[:20]

# Test set preprocessing


(Here's where encapsulating the transformations in a pipeline would come in handy. But I'll do it manually for now.)

In [ ]:
# features = [x for x in test_df.columns if x != 'claim']
# X_test = test_df[features] # this is just for naming consistency

Now, let's get the features the model was trained on and subset the test set's features accordingly

In [ ]:
# generation polynomial features
# poly = PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)
# X_test_poly = poly.fit_transform(X_test)

In [ ]:
# X_test_poly_names = poly.get_feature_names(X_test.columns)
# X_poly_names[100:150]
# features = pd.read_csv('X_candidates_20210827.csv').columns

In [ ]:
# checks = [feature in X_test_poly_names for feature in features]
# checks

In [ ]:
# X_test_final = pd.DataFrame(X_test_poly, columns=X_test_poly_names)

In [ ]:
# X_test_final = X_test_final[features[1:]]
# X_test_final = X_test

In [ ]:
# X_test['nan_count'] = X_test.isnull().sum(axis=1)

In [ ]:
# imputer = SimpleImputer(strategy='median', add_indicator=True)
# X_test_imputed_np = imputer.fit_transform(X_test)

In [ ]:
# X_test_imputed = pd.DataFrame(X_test_imputed, columns=[str(x) for x in range(X_test_imputed.shape[1])])
# X_test_imputed.to_feather(path=datapath/'X_test_NaNcounts_imputed-Median-wIndicators.feather')

In [ ]:
# scaler = exmodel_config['scaler']()
# X_test_imputed_scaled_np = scaler.fit_transform(X_test_imputed)
# X_test_imputed_scaled = pd.DataFrame(X_test_imputed_scaled_np, columns=X_test_imputed.columns)
# X_test_imputed_scaled.to_feather(path=datapath/'X_test_NaNcounts_imputed-Median-wIndicators_StandardScaled.feather')
# X_scaled_df = pd.DataFrame(X_scaled, columns=X_poly_names)

In [ ]:
test_set_path = str(datapath/'X_test_NaNcounts_SummaryStats_imputed-Median-wIndicators-StandardScaled.feather')
wandb.log({'test_set': test_set_path})

In [ ]:
# X_test_imputed_scaled = pd.read_feather(path=datapath/'X_test_NaNcounts_imputed-Median-wIndicators_StandardScaled.feather')
X_test_imputed_scaled = pd.read_feather(path=datapath/'X_test_NaNcounts_SummaryStats_imputed-Median-wIndicators-StandardScaled.feather')

## Prediction Generation

In [ ]:
preds_path = Path(datapath/"preds/")

blender_preds = blender.predict_proba(X_test_imputed_scaled)[:,1]
dump(blender_preds, preds_path/f"{config_run['name']}_stack.joblib")

# Submission

In [ ]:
sample_df = pd.read_csv(datapath/'sample_solution.csv')

In [ ]:
sample_df.loc[:, 'claim'] = blender_preds

In [ ]:
sample_df.head()

In [ ]:
submission_path = datapath/'submissions'
submission_path.mkdir(exist_ok=True)

In [55]:
sample_df.to_csv(submission_path/f"{config_run['name']}_blended.csv", index=False)

In [59]:
# str(blender.estimators[2][1].get_all_params())
# blender.estimators[2][1]

In [56]:
wandb.log({'leaderboard_auc': ,
           'catboost_params': str(best_catboost_params),
          })

In [57]:
wandb.finish()

blender-final_estimator,LogisticRegression(m...
blender-stack_mdethod,predict_proba
blender-cv,5
_runtime,16221
_timestamp,1632331738
_step,4
xgboost_params,{'objective': 'binar...
lightgbm_params,{'boosting_type': 'g...
train_loss,0.48961
train_auc,0.84839
test_set,/media/sf/easystore/...


blender-cv,▁
_runtime,▁████
_timestamp,▁████
_step,▁▃▅▆█
train_loss,▁
train_auc,▁
leaderboard_auc,▁


## Manual Stacking

In [73]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,227,228,229,230,231,232,233,234,235,236
id,,,,,,,,,,,,,,,,,,,,,
0,0.425545,-2.357891,-0.637206,-0.866657,-0.111568,-4.829243,-1.171229,-0.603397,-0.596871,-0.516828,...,-0.128368,-0.127677,-0.128242,-0.127867,-0.127119,-0.127985,-0.128494,-0.12862,-0.127854,-0.12703
1,0.247600,-0.323982,1.223569,0.361863,1.071182,-0.361140,0.082051,-0.746590,0.899454,0.469668,...,-0.128368,-0.127677,-0.128242,-0.127867,-0.127119,-0.127985,-0.128494,-0.12862,-0.127854,-0.12703
2,2.032371,-2.435680,-0.488960,0.341193,1.069656,0.118532,0.537069,-0.044075,-0.763516,1.056879,...,-0.128368,-0.127677,-0.128242,-0.127867,-0.127119,-0.127985,-0.128494,-0.12862,-0.127854,-0.12703
3,1.438373,-2.337605,-0.508914,-0.829607,1.485682,3.592008,-1.189087,-0.339152,-0.735281,-0.529158,...,-0.128368,-0.127677,-0.128242,-0.127867,-0.127119,-0.127985,-0.128494,-0.12862,-0.127854,-0.12703
4,0.602333,1.076218,-0.648438,0.463365,0.275053,-0.157989,0.727338,-0.905498,0.052478,-0.511066,...,-0.128368,-0.127677,-0.128242,-0.127867,-0.127119,-0.127985,-0.128494,-0.12862,7.821398,-0.12703


In [119]:
X1 = X.copy()

In [120]:
X1.shape

(957919, 237)

In [121]:
# generate probability predictions for the XGBoost model's folds
for fold in xgboost_models.keys():
#     X1[f"xgboost_fold{fold}_pred"] = xgboost_models[fold].predict(X)
    X1[f"xgboost_fold{fold}_pred"] = xgboost_models[fold].predict_proba(X)[:,1]
#     xgboost_preds[fold] = xgboost_models[fold].predict(X_test_imputed_scaled)

/home/sf/anaconda3/envs/tabular/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [122]:
X1.head()

,0,1,2,3,4,5,6,7,8,9,...,232,233,234,235,236,xgboost_fold0_pred,xgboost_fold1_pred,xgboost_fold2_pred,xgboost_fold3_pred,xgboost_fold4_pred
id,,,,,,,,,,,,,,,,,,,,,
0,0.425545,-2.357891,-0.637206,-0.866657,-0.111568,-4.829243,-1.171229,-0.603397,-0.596871,-0.516828,...,-0.127985,-0.128494,-0.12862,-0.127854,-0.12703,0.582566,0.580950,0.576743,0.569523,0.595877
1,0.247600,-0.323982,1.223569,0.361863,1.071182,-0.361140,0.082051,-0.746590,0.899454,0.469668,...,-0.127985,-0.128494,-0.12862,-0.127854,-0.12703,0.152252,0.150803,0.148316,0.155218,0.147297
2,2.032371,-2.435680,-0.488960,0.341193,1.069656,0.118532,0.537069,-0.044075,-0.763516,1.056879,...,-0.127985,-0.128494,-0.12862,-0.127854,-0.12703,0.794083,0.789945,0.788326,0.787177,0.797979
3,1.438373,-2.337605,-0.508914,-0.829607,1.485682,3.592008,-1.189087,-0.339152,-0.735281,-0.529158,...,-0.127985,-0.128494,-0.12862,-0.127854,-0.12703,0.774001,0.768510,0.774555,0.782187,0.773245
4,0.602333,1.076218,-0.648438,0.463365,0.275053,-0.157989,0.727338,-0.905498,0.052478,-0.511066,...,-0.127985,-0.128494,-0.12862,7.821398,-0.12703,0.759366,0.755764,0.763769,0.758034,0.758038
